In [ ]:
!pip install --upgrade diffusers transformers scipy
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 503 kB 7.4 MB/s 
     |████████████████████████████████| 5.8 MB 62.0 MB/s 
     |████████████████████████████████| 33.8 MB 1.3 MB/s 
     |████████████████████████████████| 182 kB 81.7 MB/s 
     |████████████████████████████████| 7.6 MB 37.6 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 191 kB 7.4 MB/s 


### Hugging face token. To create one, follow the provided link and create a token with the "write" access

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
    
Token: 
Add token as git credential? (Y/n) n
Token is valid.
Your token has been saved to /root/.huggingface/token
Login successful


## Model that was trained with the specialized fast-DreamBooth fast stable diffusion tuner. [Github](https://github.com/huggingface/diffusers). Model is then saved onto Huggingface

In [ ]:
import torch
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained("oacev/heightmapsstyle", torch_dtype=torch.float16)
pipe = pipeline.to("cuda")

Downloading:   0%|          | 0.00/543 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/342 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/931 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/611 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335M [00:00<?, ?B/s]

##Enter your prompt here
example: "Heightmap of a mountain range with a ridge going through the center"

In [ ]:
prompt = "heightmap of a valley where there was a crater in the center"

## Seperate lines here due to CUDA GPU avaliability

In [ ]:
image = pipe(prompt).images[0]
image.save("sample.png")

  0%|          | 0/50 [00:00<?, ?it/s]

## Image processing and heightmap mapping


In [ ]:
import numpy as np
from PIL import Image, ImageOps

In [ ]:
MAP_SIZE = (512,512)  # Should be same as image size: 256 x 256
MAP = np.zeros(MAP_SIZE)

im = Image.open("sample.png") 
inverted = False

if inverted == True:
  im = im.convert('L')
  im_invert = ImageOps.invert(im)
  im_invert.save('sample_invert.png')
  im.save('sample_bnw.png')
  im = Image.open('sample_invert.png') 
  

jpg_im = im.convert('RGB')
pix = jpg_im.load()

for y in range(512):
  for x in range(512):
    MAP[y,x] = pix[x,y][0]

MAP_norm = (MAP-np.min(MAP))/(np.max(MAP)-np.min(MAP))

# Object Generation

In [ ]:
# simple version of height map generation from https://github.com/LoaDy588/py_terrain_mesh 

def generate_vertices_triangles(heightmap,MAP_SIZE):
    vertices = []
    base = (-1, -0.75, -1)
    size = 2
    max_height = 0.5
    step_x = size/(MAP_SIZE[0]-1)
    step_y = size/(MAP_SIZE[1]-1)

    for x in range(MAP_SIZE[0]):
        for y in range(MAP_SIZE[1]):
            x_coord = base[0] + step_x*x 
            y_coord = base[1] + max_height*heightmap[x][y]
            z_coord = base[2] + step_y*y
            vertices.append((x_coord, y_coord, z_coord))

    tris = []
    for x in range(MAP_SIZE[0]-1):
        for y in range(MAP_SIZE[1]-1):
            index = x*MAP_SIZE[0]+y
            a = index
            b = index+1
            c = index+MAP_SIZE[0]+1
            d = index+MAP_SIZE[0]
            tris.append((a, b, c))
            tris.append((a, c, d))

    return (vertices, tris)

def export_obj(vertices, tris, filename):
    file = open(filename, "w")
    print("running...")
    for vertex in vertices:
      file.write("v " + str(vertex[0]) + " " + str(vertex[1]) + " " + str(vertex[2]) + "\n")
    print("vertices done...")
    for tri in tris:
      file.write("f " + str(tri[2]+1) + " " + str(tri[1]+1) + " " + str(tri[0]+1) + "\n")
    print("completed")
    file.close()
    return

def generateOBJ(MAP_SIZE, MAP_norm, name):
    vertices, tris = generate_vertices_triangles(MAP_norm, MAP_SIZE)
    export_obj(vertices, tris, name)

In [ ]:
generateOBJ(MAP_SIZE, MAP_norm, "sample.obj")

running...
vertices done...
completed
